In [1]:
pnt_file = "/home/benr/wqmodels/ssm/loadings/cequalicm_wq_2014_Exist3_v3.dat"
start_date = '2014-01-01'
out_file_nc = "data/ssm_pnt_2014_v3.nc"
out_file_excel = "data/ssm_pnt_2014_v3.xlsx"

from io import StringIO
import os
import shutil
import numpy as np
import pandas as pd
from netCDF4 import Dataset

In [2]:
with open(pnt_file) as f:
    # The parsing logic here is is derived from the linkage instructions for the
    # model and direct examination of the source code
    
    # The first line does not contain important information and is treated only like
    # a filetype magic
    next(f)

    # The total number of discharge nodes
    num_qs = int(next(f))
    # All the node numbers with discharges
    #nodes = np.loadtxt([next(f) for l in range(num_qs)], comments='!', dtype=int)
    node_raw = StringIO('\n'.join([next(f) for l in range(num_qs)]))
    node_df = pd.read_csv(node_raw, sep='\s+!\s+', names=('Node','Comment'),
                          dtype={'Node':np.int64,'Comment':object}, engine='python')
    node_df.set_index('Node', inplace=True)
    nodes = node_df.index.to_numpy()
    # Depth distribution fractions into each node. Skipping the first (node count) column
    vqdist = np.loadtxt([next(f) for l in range(num_qs)])[:,1:]

    num_times = int(next(f))

    # Initialize storage arrays
    times = np.zeros(num_times)
    qs = np.zeros((num_times, num_qs))
    # State variables in the order they are present in the file. These are also going
    # to be the NetCDF variable names
    statevars = ('discharge', 'temp', 'salt', 'tss',  'alg1', 'alg2', 'alg3', 'zoo1',
                              'zoo2', 'ldoc', 'rdoc', 'lpoc', 'rpoc', 'nh4',  'no32',
                              'urea', 'ldon', 'rdon', 'lpon', 'rpon', 'po4',  'ldop',
                              'rdop', 'lpop', 'rpop', 'pip',  'cod',  'doxg', 'psi',
                              'dsi',  'alg1p','alg2p','alg3p','dic',  'talk')
    statedata = {}
    for v in statevars:
        statedata[v] = np.zeros((num_times, num_qs))

    for t in range(num_times):
        times[t] = float(next(f))
        for v in statevars:
            statedata[v][t,:] = np.loadtxt([next(f)])

print("Times:", times.shape)
print("Nodes:", nodes.shape)
print("VQdist:", vqdist.shape)
print("NO32:", statedata['no32'].shape)
statedata['no32']

Times: (366,)
Nodes: (259,)
VQdist: (259, 10)
NO32: (366, 259)


array([[1.679e-01, 1.753e-01, 2.028e+00, ..., 2.500e-02, 1.340e+01,
        0.000e+00],
       [1.713e-01, 1.790e-01, 2.022e+00, ..., 2.500e-02, 1.340e+01,
        3.235e-03],
       [2.142e-01, 2.236e-01, 2.007e+00, ..., 2.500e-02, 1.340e+01,
        2.416e-02],
       ...,
       [3.183e-01, 3.316e-01, 1.600e+00, ..., 2.000e-02, 1.486e+01,
        7.002e-02],
       [3.024e-01, 3.151e-01, 1.596e+00, ..., 2.000e-02, 1.486e+01,
        6.981e-02],
       [3.119e-01, 3.250e-01, 1.620e+00, ..., 4.400e-02, 9.823e+00,
        6.804e-02]])

In [3]:
cdf = Dataset(out_file_nc, "w")

time_dim = cdf.createDimension("time", len(times))
node_dim = cdf.createDimension("node", num_qs)
siglay_dim = cdf.createDimension("siglay", vqdist.shape[1])

time_var = cdf.createVariable("time", "f4", ("time",))
time_var.unit = "hours"
cdf['time'][:] = times
node_var = cdf.createVariable("node", "i4", ("node",))
cdf['node'][:] = nodes
vqdist_var = cdf.createVariable("vqdist", "f4", ("node","siglay"))
cdf['vqdist'][:] = vqdist

for v in statevars:
    var = cdf.createVariable(v, "f4", ("time","node"))
    cdf[v][:] = statedata[v]

cdf

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): time(366), node(259), siglay(10)
    variables(dimensions): float32 time(time), int32 node(node), float32 vqdist(node, siglay), float32 discharge(time, node), float32 temp(time, node), float32 salt(time, node), float32 tss(time, node), float32 alg1(time, node), float32 alg2(time, node), float32 alg3(time, node), float32 zoo1(time, node), float32 zoo2(time, node), float32 ldoc(time, node), float32 rdoc(time, node), float32 lpoc(time, node), float32 rpoc(time, node), float32 nh4(time, node), float32 no32(time, node), float32 urea(time, node), float32 ldon(time, node), float32 rdon(time, node), float32 lpon(time, node), float32 rpon(time, node), float32 po4(time, node), float32 ldop(time, node), float32 rdop(time, node), float32 lpop(time, node), float32 rpop(time, node), float32 pip(time, node), float32 cod(time, node), float32 doxg(time, node), float32 psi(time, node), float32 ds

In [4]:
cdf.close()

Save the output in the form of an Excel spreadsheet which is more easily editable.

Three worksheets will be created. The first will have the node list and any comments parsed from the original file, which generally says what feature the node corresponds to. The second is the vertical distribution of inflow into the node. The third is all of the flow and water quality data for all nodes at all dates.

In [5]:
dates = pd.Timestamp(start_date) + pd.to_timedelta(times, 'h')
dates.name = 'Date'
with pd.ExcelWriter(out_file_excel) as writer:
    node_df.to_excel(writer, sheet_name='Nodes')
    pd.DataFrame(vqdist, index=node_df.index, columns=np.arange(vqdist.shape[1])+1).to_excel(writer, sheet_name='VQDist')

    node_data = []
    for i,n in enumerate(nodes):
        n_fill = np.zeros(len(times), dtype=np.int64) + n
        df = pd.DataFrame({v: statedata[v][:,i] for v in statevars}, index=[dates,pd.Index(n_fill,name='Node')])
        node_data.append(df)
    node_data_df = pd.concat(node_data)
    display(node_data_df)
    node_data_df.to_excel(writer, sheet_name='Data')

,,discharge,temp,salt,tss,alg1,alg2,alg3,zoo1,zoo2,ldoc,...,pip,cod,doxg,psi,dsi,alg1p,alg2p,alg3p,dic,talk
Date,Node,,,,,,,,,,,,,,,,,,,,,
2014-01-01,15634,0.1528,5.939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4460,...,0.0,0.0,11.49,0.0,0.0,0.0,0.0,0.0,547.1,440.0
2014-01-02,15634,0.1548,5.939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4010,...,0.0,0.0,11.49,0.0,0.0,0.0,0.0,0.0,547.6,440.0
2014-01-03,15634,0.2051,5.939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3310,...,0.0,0.0,11.42,0.0,0.0,0.0,0.0,0.0,555.1,440.0
2014-01-04,15634,0.1683,5.939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3100,...,0.0,0.0,11.49,0.0,0.0,0.0,0.0,0.0,549.7,440.0
2014-01-05,15634,0.1528,5.939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.2750,...,0.0,0.0,11.53,0.0,0.0,0.0,0.0,0.0,547.4,440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-28,13429,43.5100,6.784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5170,...,0.0,0.0,11.71,0.0,0.0,0.0,0.0,0.0,536.2,451.4
2014-12-29,13429,46.8600,6.784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5414,...,0.0,0.0,11.74,0.0,0.0,0.0,0.0,0.0,536.8,451.4
2014-12-30,13429,50.4800,6.784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5653,...,0.0,0.0,11.76,0.0,0.0,0.0,0.0,0.0,537.4,451.4
